In [20]:
import pandas as pd
import us

In [28]:
state_names = [state.name for state in us.states.STATES]

In [17]:
df = pd.read_csv('Retail_sales_of_electricity.csv', index_col=0, skiprows=4, na_values='--')
df = df.drop(['Retail sales of electricity', 'United States']).drop(['source key', 'units'], axis=1)
df = df.drop([nm for nm in df.index if ':' not in nm], axis=0)
df = df.T

In [23]:
df = df.rename(columns={c: c.split(':')[0].strip() for c in df.columns if ': all sectors' in c})

In [29]:
customers = pd.read_excel('https://www.eia.gov/electricity/sales_revenue_price/xls/table5_a.xlsx', skiprows=2).set_index('State')['Number of Customers'][state_names]

In [59]:
df.head()

description,United States,United States : residential,United States : commercial,United States : industrial,United States : transportation,United States : other,New England,New England : residential,New England : commercial,New England : industrial,...,Alaska : commercial,Alaska : industrial,Alaska : transportation,Alaska : other,Hawaii,Hawaii : residential,Hawaii : commercial,Hawaii : industrial,Hawaii : transportation,Hawaii : other
2001,3394458.0,1201607.0,1083069.0,996609.0,NaN,113174.0,118483.0,42359.0,49424.0,25219.0,...,2289.0,1079.0,NaN,194.0,9785.0,2802.0,3129.0,3790.0,NaN,63.0
2002,3465466.0,1265180.0,1104497.0,990238.0,NaN,105552.0,119728.0,44089.0,49939.0,24152.0,...,2238.0,1088.0,NaN,207.0,9892.0,2898.0,3168.0,3770.0,NaN,55.0
2003,3493734.0,1275824.0,1198728.0,1012373.0,6810.0,NaN,123438.0,46250.0,52389.0,24315.0,...,2473.0,1104.0,0.0,NaN,10391.0,3028.0,3517.0,3846.0,0.0,NaN
2004,3547479.0,1291982.0,1230425.0,1017850.0,7224.0,NaN,125249.0,46703.0,53683.0,24267.0,...,2601.0,1126.0,0.0,NaN,10732.0,3162.0,3632.0,3937.0,0.0,NaN
2005,3660969.0,1359227.0,1275079.0,1019156.0,7506.0,NaN,127862.0,48701.0,54777.0,23792.0,...,2695.0,1156.0,0.0,NaN,10539.0,3164.0,3463.0,3912.0,0.0,NaN


In [63]:
# average residential monthly kWh
avg_montly_use = (df[[f"{s} : residential" for s in state_names]].rename(columns={f"{s} : residential": s for s in state_names}) * 1e6 / 12).div(customers[state_names], axis=1).round()

## percent change 

In [76]:
pct_change = (((avg_montly_use.loc['2019'] - avg_montly_use.loc['2010']).div(avg_montly_use.loc['2010'], axis=0)) * 100).round(1).sort_values()

In [78]:
pct_change.head()

description
West Virginia   -10.4
Massachusetts   -10.3
Kentucky         -8.4
Alabama          -8.3
Hawaii           -7.7
dtype: float64

In [79]:
pct_change.tail()

description
Nevada          10.5
Montana         10.7
Maine           10.9
Texas           11.5
North Dakota    14.7
dtype: float64

## ranks

In [66]:
pd.DataFrame({
    year: avg_montly_use.loc[str(year)].sort_values(ascending=False).head(10).rank(ascending=False)
    for year in [2010, 2019]
}).sort_values([2019, 2010], ascending=True)

,2010,2019
Louisiana,2.5,1.0
Tennessee,2.5,2.0
Mississippi,4.0,3.0
Alabama,1.0,4.0
Texas,NaN,5.0
South Carolina,7.0,6.0
Georgia,8.0,7.0
Oklahoma,NaN,8.0
Virginia,9.0,9.0
Arkansas,10.0,10.0


In [67]:
pd.DataFrame({
    year: avg_montly_use.loc[str(year)].sort_values(ascending=True).head(10).rank(ascending=True)
    for year in [2010, 2019]
}).sort_values([2010, 2019], ascending=True)

,2010,2019
Maine,1.0,6.0
California,2.0,2.0
Vermont,3.0,3.0
Hawaii,4.0,1.0
Rhode Island,5.0,4.0
New York,6.0,8.0
New Hampshire,7.0,9.0
Alaska,8.0,5.0
New Mexico,9.0,NaN
Massachusetts,10.0,7.0
